<a href="https://colab.research.google.com/github/mynhungg/Datamining/blob/main/Project_PhoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import thư viện



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd




##1.   Load model BERT





In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount=True)
path = "/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4"
os.chdir(path)
!ls

Mounted at /content/gdrive


FileNotFoundError: ignored



1.   Cài package
*   fairseq
*   fastBPE
*   vncorenlp
*   transformers





In [ ]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



2.   Download model pretrain từ [PhoBERT](https://github.com/VinAIResearch/PhoBERT)

Project sử dụng pretrain model BERT base được huấn luyện từ package fairseq.


In [ ]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2023-05-23 08:13:16--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.150.44, 54.192.150.106, 54.192.150.17, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.150.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  45.9MB/s    in 25s     

2023-05-23 08:13:41 (48.4 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [ ]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2023-05-23 08:14:12--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.150.106, 54.192.150.98, 54.192.150.44, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.150.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  57.2MB/s    in 6.2s    

2023-05-23 08:14:18 (49.9 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


Sau khi download và giải nén pretrain file chúng ta sẽ kiểm tra thấy bên trong folder sẽ bao gồm 3 files đó là `bpe.codes, dict.txt, model.pt` có tác dụng như sau:

* bpe.codes: Là BPE token mà mô hình đã áp dụng để mã hóa văn bản sang index.

* dict.txt: Từ điển của bộ dữ liệu huấn luyện.

* model.pt: File lưu trữ của mô hình trên pytorch.

In [ ]:
!ls PhoBERT_base_fairseq

bpe.codes  dict.txt  model.pt


## 2. Load model pretrain PhoBERT

In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

Ta có thể thấy kiến trúc RoBERTa đã giữa lại 12 block sub-layers là các multi-head attention ở phase Encoder và thêm một linear projection layer ở cuối để tạo ra một embedding cho từ hiện tại.

**Áp dụng BPE tokenize trong BERT**

In [ ]:
!pip3 install fairseq
!pip3 install fastBPE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load model pretrain `RoBERTa`.

In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

Khai báo bpe tokenizer và thực hiện token.

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE

# Khởi tạo Byte Pair Encoding cho PhoBERT
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
tokens = phoBERT.encode('Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh')
print('tokens list : ', tokens)
# Decode ngược lại thành câu từ chuỗi index token
phoBERT.decode(tokens)  # 'Hello world!'

tokens list :  tensor([    0, 11623, 31433,   453, 44334,  2080,  5922,    57,   934,  8181,
        31686,  3078,     2])


'Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh'

Down load package VnCoreNLP để tokenize các câu văn.

In [ ]:

!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2023-05-23 08:15:25--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  59.1MB/s    in 0.4s    

2023-05-23 08:15:28 (59.1 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2023-05-23 08:15:28--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

Gỉa sử chúng ta có câu gốc là `Tôn Ngộ Không phò Đường Tăng đi thỉnh kinh tại Tây Trúc`. Từ được ẩn đi trong câu là `phò` sẽ được thay thế bằng token `<mask>`.

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = 'Tôn Ngộ Không phò Đường Tăng đi thỉnh kinh tại Tây Trúc'
text_masked = 'Học sinh được  <mask> do dịch covid-19'
# Tokenize câu gốc và thay từ phò bằng <mask>
words = rdrsegmenter.tokenize(text)[0]
for i, token in enumerate(words):
  if token == 'phò':
    words[i] = ' <mask>'
text_masked_tok = ' '.join(words)
print('text_masked_tok: \n', text_masked_tok)

text_masked_tok: 
 Tôn_Ngộ_Không  <mask> Đường Tăng đi thỉnh_kinh tại Tây_Trúc


Tìm ra top 10 từ thích hợp nhất cho vị trí `<mask>` tại câu trên.

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
import numpy as np

# Khởi tạo Byte Pair Encoding cho PhoBERT
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Filling marks  
topk_filled_outputs = phoBERT.fill_mask(text_masked_tok, topk=10) 
topk_probs = [item[1] for item in topk_filled_outputs]
print('Total probability: ', np.sum(topk_probs))
print('Input sequence: ', text_masked_tok)
print('Top 10 in mask: ')
for i, output in enumerate(topk_filled_outputs): 
  print(output[0])

Total probability:  0.8735206946730614
Input sequence:  Tôn_Ngộ_Không  <mask> Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Top 10 in mask: 
Tôn_Ngộ_Không và Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không đưa Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không cõng Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không hộ_tống Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không cùng Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không chở Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không theo Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không dẫn Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không , Đường Tăng đi thỉnh_kinh tại Tây_Trúc
Tôn_Ngộ_Không tháp_tùng Đường Tăng đi thỉnh_kinh tại Tây_Trúc


## 7. Extract feature cho các từ

Chúng ta có thể tìm ra được các véc tơ embedding cho từng từ trong câu từ mô hình BERT như sau:

In [ ]:
#from fairseq.data.encoders.fastbpe import fastBPE

# Khởi tạo Byte Pair Encoding cho PhoBERT
#class BPE():
  #bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

#args = BPE()
#phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
#doc = phoBERT.extract_features_aligned_to_words('học_sinh cấp 3 được đến trường sau nghỉ dịch covid')

#for tok in doc:
 #   print('{:10}{} (...) {}'.format(str(tok), tok.vector[:5], tok.vector.size()))

## 8. Bài toán classification

### 8.2. Dữ liệu

sử dụng dữ liệu [VNTC](https://github.com/duyvuleo/VNTC.git) với các bài báo đã được sắp xếp theo 10 topics. Bộ dữ liệu bao gồm 33 nghìn bài báo trên tập train và 50 nghìn bài báo trên tập test có phân bố số lượng theo topics như sau:

![](https://imgur.com/1lDTdC1.png)

8.2.1. Đọc và lưu dữ liệu

In [ ]:
!chmod +x /content/gdrive/MyDrive/Colab\ Notebooks/PhoBERT_VNTC_4/VNTC/.git/hooks/post-checkout


chmod: cannot access '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/.git/hooks/post-checkout': No such file or directory


In [ ]:
!git clone https://github.com/duyvuleo/VNTC.git
!ls VNTC/Data/10Topics/Ver1.1

Cloning into 'VNTC'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 160.90 MiB | 5.51 MiB/s, done.
Updating files: 100% (15/15), done.
Filtering content: 100% (2/2), 168.95 MiB | 19.76 MiB/s, done.
fatal: cannot exec '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/.git/hooks/post-checkout': Permission denied
Stats.txt  Test_Full.rar  Train_Full.rar


Extract file dataset

In [ ]:

!pip install patool
import patoolib
# Specify the path to the RAR file
rar_file_path = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Train_Full.rar'

# Specify the destination directory for extraction
destination_dir = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'

# Extract the RAR file
patoolib.extract_archive(rar_file_path, outdir=destination_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
patool: Extracting /content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Train_Full.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Train_Full.rar"
patool:     with cwd='/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'
patool: ... /content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Train_Full.rar extracted to `/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'.


'/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'

In [ ]:
# Specify the path to the RAR file
rar_file_path = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Test_Full.rar'

# Specify the destination directory for extraction
destination_dir = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'

# Extract the RAR file
patoolib.extract_archive(rar_file_path, outdir=destination_dir)

patool: Extracting /content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Test_Full.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Test_Full.rar"
patool:     with cwd='/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'
patool: ... /content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Test_Full.rar extracted to `/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'.


'/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1'

Sau khi đã download dữ liệu về, chúng ta sẽ đọc và lưu các bài báo vào những list chứa nội dung và nhãn tương ứng theo 2 folders train và test.

In [ ]:
import glob2
from tqdm import tqdm

train_path = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Train_Full/*/*.txt'
test_path = '/content/gdrive/MyDrive/Colab Notebooks/PhoBERT_VNTC_4/VNTC/Data/10Topics/Ver1.1/Test_Full/*/*.txt'

# Hàm đọc file txt
def read_txt(path):
  with open(path, 'r', encoding='utf-16') as f:
    data = f.read()
  return data

# Hàm tạo dữ liệu huấn luyện cho tập train và test
def make_data(path):
  texts = []
  labels = []
  for file_path in tqdm(glob2.glob(train_path)):
    try:
      content = read_txt(file_path)
      label = file_path.split('/')[11]
      texts.append(content)
      labels.append(label)
    except:
      next
  return texts, labels

text_train, label_train = make_data(train_path)
text_test, label_test = make_data(test_path)

100%|██████████| 33759/33759 [00:41<00:00, 817.85it/s]


Tạo các hàm lưu trữ lại các list nội dung và nhãn và load lại cho lượt huấn luyện sau.

In [ ]:
import pickle

def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)



In [ ]:
def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj


In [ ]:
# Lưu lại các files
_save_pkl('text_train.pkl', text_train)
_save_pkl('label_train.pkl', label_train)
_save_pkl('text_test.pkl', text_test)
_save_pkl('label_test.pkl', label_test)

In [ ]:
print('text content:\n', text_train[0])
print('label:\n', label_train[0])

text content:
 Các nhà khoa học nói về ô nhiễm tại hố chôn gà dịch 
Trong những ngày qua, có nhiều ý kiến phản ánh của người dân về tình trạng ô nhiễm tại các điểm chôn gà chết của TP HCM. Ở các hố chôn, nước dịch trào ra, thoát khí nặng mùi, nhất là vào những hôm trời nóng bức. VnExpress đã trao đổi với một số nhà khoa học và quản lý môi trường về vấn đề này.
Giáo sư, Tiến sĩ Tưởng Thị Hội, giảng viên Viện Khoa học và công nghệ Môi trường, ĐH Bách Khoa Hà Nội: Động vật khi chôn sâu trong lòng đất sẽ bị phân hủy dưới dạng hiếu khí và yếm khí, tạo ra những hợp chất dạng khác nhau như thể lỏng hoặc thể khí. Các dịch lỏng phát sinh có thể ngấm vào đất và gây ô nhiễm nguồn nước. Điều này trước kia cũng đã xảy ra ở nghĩa trang Văn Điển, Hà Nội. Các khí tạo thành trong quá trình phân hủy chất hữu cơ như CH4, NH3, H2S, CO2 thoát ra ngoài môi trường sẽ tạo ra mùi hôi, gây độc hại đến môi trường sống.
Giáo sư Nguyễn Công Mẫn, Phó chủ tịch Viện Địa kỹ thuật thuộc Liên hiệp các Hội khoa học kỹ th

#### 8.2.2. Tokenize nội dung

Tiếp theo ta sẽ tokenize các câu văn sang chuỗi index và padding câu văn về cùng một độ dài.

In [ ]:
max_sequence_length = 500

def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length)) # --> shape (number_lines, max_seq_len)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
    
    outputs[idx,:] = np.array(input_ids)
  return outputs

### 8.3. Tokenize Input và output

* Chuẩn bị X input: Tokenize nội dung các văn bản sang chuỗi indices.

* Chuẩn bị y output: Encoding các label output thành indices đánh dấu số thứ tự của văn bản.

In [ ]:
from tqdm import tqdm
import torch

max_sequence_length = 256
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
    
    outputs[idx,:] = np.array(input_ids)
  return outputs


In [ ]:
from fairseq.data import Dictionary

In [ ]:

# Load the dictionary  
vocab = Dictionary()
vocab.add_from_file("PhoBERT_base_transformers/dict.txt")


In [ ]:

# Test encode lines
lines = ['Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19', 'số lượng ca nhiễm bệnh đã giảm bắt đầu từ tháng 5 nhờ biện pháp mạnh tay']
[x1, x2] = convert_lines(lines, vocab, phoBERT.bpe)
print('x1 tensor encode: {}, shape: {}'.format(x1[:10], x1.size))
print('x1 tensor decode: ', phoBERT.decode(torch.tensor(x1))[:103])

100%|██████████| 2/2 [00:00<00:00, 691.39it/s]

x1 tensor encode: [63117  1302   884  5958    11   915   222   537   933    39], shape: 256
x1 tensor decode:  <s> Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19 </s> <pad> <pad> <pad> <pad> <pad>


Như vậy ta thấy rằng các câu văn đã được encode về token index. Từ token index có thể decode ngược trở lại thành câu input sau khi đã thêm các token đặc biệt đánh dấu vị trí bắt dầu: `<s>`, kết thúc: `</s>` câu và các vị trí nằm ngoài câu: `<pad>`. Ta sẽ token toàn bộ câu input sang index như sau:

In [ ]:
X = convert_lines(text_train, vocab, phoBERT.bpe)
print('X shape: ', X.shape)

100%|██████████| 33759/33759 [02:45<00:00, 204.19it/s]

X shape:  (33759, 256)


Sau cùng ta thu được các chuỗi index có kích thước là 256, bằng với kích thước của các câu sau khi đã padding. Tiếp theo ta tạo output y bằng index cho các nhãn của câu.

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(label_train)
y = lb.fit_transform(label_train)
print(lb.classes_)
print('Top 5 classes indices: ', y[:5])

['Chinh tri Xa hoi' 'Doi song' 'Khoa hoc' 'Kinh doanh' 'Phap luat'
 'Suc khoe' 'The gioi' 'The thao' 'Van hoa' 'Vi tinh']
Top 5 classes indices:  [0 0 0 0 0]


Lưu lại dữ liệu $\mathbf{X}$ và $\mathbf{y}$

In [ ]:
# Save dữ liệu
_save_pkl('PhoBERT_pretrain/X1.pkl', X)
_save_pkl('PhoBERT_pretrain/y1.pkl', y)
_save_pkl('PhoBERT_pretrain/labelEncoder1.pkl', lb)

# Load lại dữ liệu
X = _load_pkl('PhoBERT_pretrain/X1.pkl')
y = _load_pkl('PhoBERT_pretrain/y1.pkl')

print('length of X: ', len(X))
print('length of y: ', len(y))

length of X:  33759
length of y:  33759


### 8.4. Load model BERT

In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

phoBERT_cls = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

# Load BPE
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Add header cho classification với số lượng classes = 10
phoBERT_cls.register_classification_head('new_task', num_classes=10)
tokens = 'Học_sinh được nghỉ học bắt đầu từ tháng 3 do ảnh hưởng của dịch covid-19'
token_idxs = phoBERT_cls.encode(tokens)
logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
logprobs

tensor([[-2.2793, -2.2856, -2.0900, -2.0557, -2.4173, -2.4040, -2.3832, -2.4373,
         -2.3396, -2.4223]], grad_fn=<LogSoftmaxBackward0>)

Xây dựng hàm đánh giá mô hình theo 2 metric là accuracy và f1_score.

### 8.5. Huấn luyện model

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def evaluate(logits, targets):
    """
    Đánh giá model sử dụng accuracy và f1 scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho class output.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        acc (float): the accuracy score
        f1 (float): the f1 score
    """
    # Tính accuracy score và f1_score
    logits = logits.detach().cpu().numpy()    
    y_pred = np.argmax(logits, axis = 1)
    targets = targets.detach().cpu().numpy()
    f1 = f1_score(targets, y_pred, average='weighted')
    acc = accuracy_score(targets, y_pred)
    return acc, f1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)
targets = torch.tensor([1, 2, 2]).to(device)
evaluate(logits, targets)

(0.6666666666666666, 0.5333333333333333)

In [ ]:
def validate(valid_loader, model, device):
    model.eval()
    accs = []
    f1s = []
    with torch.no_grad():
        for x_batch, y_batch in valid_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model.predict('new_task', x_batch)
            logits = torch.exp(outputs)
            acc, f1 = evaluate(logits, y_batch)
            accs.append(acc)
            f1s.append(f1)
    
    mean_acc = np.mean(accs)
    mean_f1 = np.mean(f1s)
    return mean_acc, mean_f1

Hàm huấn luyện mô hình trên từng epoch.

In [ ]:
def trainOnEpoch(train_loader, model, optimizer, epoch, num_epochs, criteria, device, log_aggr = 100):
    model.train()
    sum_epoch_loss = 0
    sum_acc = 0
    sum_f1 = 0
    start = time.time()
    for i, (x_batch, y_batch) in enumerate(train_loader):
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      optimizer.zero_grad()
      y_pred = model.predict('new_task', x_batch)
      logits = torch.exp(y_pred)
      acc, f1 = evaluate(logits, y_batch)
      loss = criteria(y_pred, y_batch)
      loss.backward()
      optimizer.step()

      loss_val = loss.item()
      sum_epoch_loss += loss_val
      sum_acc += acc
      sum_f1 += f1
      iter_num = epoch * len(train_loader) + i + 1

      if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f),  avg acc: %.4f, avg f1: %.4f, (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),  sum_acc/(i+1), sum_f1/(i+1),
                  len(x_batch) / (time.time() - start)))
      start = time.time()  

Quá trình huấn luyện một model classification trên pytorch sẽ bao gồm những bước chính sau đây:

* Khởi tạo DataLoader để quản lý dữ liệu đưa vào huấn luyện và thẩm định.

* Thiết lập kiến trúc mô hình.

* Khai báo hàm loss function.

* Phương pháp optimization giúp tối ưu loss function.

* Huấn luyện mô hình qua các epochs.

Bên dưới chúng ta sẽ lần lượt thực hiện các bước trên.

In [ ]:
import os
import time
import random
import argparse
import pickle
import numpy as np
from tqdm import tqdm
from os.path import join

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from sklearn.model_selection import StratifiedKFold


* Khởi tạo DataLoader để quản lý dữ liệu đưa vào huấn luyện và thẩm định.

In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from transformers.modeling_utils import * 
from transformers import *

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

In [ ]:
# Khởi tạo argument
EPOCHS = 5
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 4
LR = 0.0001
LR_DC_STEP = 80 
LR_DC = 0.1
CUR_DIR = os.path.dirname(os.getcwd())
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4
CKPT_PATH2 = 'model_ckpt2'

if not os.path.exists(CKPT_PATH2):
    os.mkdir(CKPT_PATH2)

In [ ]:
# Khởi tạo DataLoader
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X, y))

for fold, (train_idx, val_idx) in enumerate(splits):
    best_score = 0
    if fold != FOLD:
        continue
    print("Training for fold {}".format(fold))
    
    # Create dataset
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))

    # Create DataLoader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Khởi tạo model:
    MODEL_LAST_CKPT = os.path.join(CKPT_PATH2, 'latest_checkpoint.pth.tar')
    if os.path.exists(MODEL_LAST_CKPT):
      print('Load checkpoint model!')
      phoBERT_cls = torch.load(MODEL_LAST_CKPT)
    else:
      print('Load model pretrained!')
      # Load the model in fairseq
      from fairseq.models.roberta import RobertaModel
      from fairseq.data.encoders.fastbpe import fastBPE
      from fairseq.data import Dictionary

      phoBERT_cls = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
      phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

      # # Load BPE
      # class BPE():
      #   bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

      # args = BPE()
      # phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

      # Add header cho classification với số lượng classes = 10
      phoBERT_cls.register_classification_head('new_task', num_classes=10)
      
    ## Load BPE
    print('Load BPE')
    class BPE():
      bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

    args = BPE()
    phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
    phoBERT_cls.to(DEVICE)

    # Khởi tạo optimizer và scheduler, criteria
    print('Init Optimizer, scheduler, criteria')
    param_optimizer = list(phoBERT_cls.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    num_train_optimization_steps = int(EPOCHS*len(train_dataset)/BATCH_SIZE/ACCUMULATION_STEPS)
    optimizer = AdamW(optimizer_grouped_parameters, lr=LR, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # scheduler với linear warmup
    scheduler0 = get_constant_schedule(optimizer)  # scheduler với hằng số
    # optimizer = optim.Adam(phoBERT_cls.parameters(), LR)
    criteria = nn.NLLLoss()
    # scheduler = StepLR(optimizer, step_size = LR_DC_STEP, gamma = LR_DC)
    avg_loss = 0.
    avg_accuracy = 0.
    frozen = True
    for epoch in tqdm(range(EPOCHS)):
        # warm up tại epoch đầu tiên, sau epoch đầu sẽ phá băng các layers
        if epoch > 0 and frozen:
            for child in phoBERT_cls.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            del scheduler0
            torch.cuda.empty_cache()
        # Train model on EPOCH
        print('Epoch: ', epoch)
        trainOnEpoch(train_loader=train_loader, model=phoBERT_cls, optimizer=optimizer, epoch=epoch, num_epochs=EPOCHS, criteria=criteria, device=DEVICE, log_aggr=100)
        # scheduler.step(epoch = epoch)
        # Phá băng layers sau epoch đầu tiên
        if not frozen:
            scheduler.step()
        else:
            scheduler0.step()
        optimizer.zero_grad()
        # Validate on validation set
        acc, f1 = validate(valid_loader, phoBERT_cls, device=DEVICE)
        print('Epoch {} validation: acc: {:.4f}, f1: {:.4f} \n'.format(epoch, acc, f1))

        # Store best model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': phoBERT_cls.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # Save model checkpoint into 'latest_checkpoint.pth.tar'
        torch.save(ckpt_dict, MODEL_LAST_CKPT)

Training for fold 4
Load model pretrained!
Load BPE


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Init Optimizer, scheduler, criteria


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:  0
[TRAIN] epoch 1/5  observation 0/4502 batch loss: 2.2823 (avg 2.2823),  avg acc: 0.0000, avg f1: 0.0000, (1.92 im/s)
[TRAIN] epoch 1/5  observation 100/4502 batch loss: 2.1053 (avg 2.3850),  avg acc: 0.1287, avg f1: 0.0626, (21.60 im/s)
[TRAIN] epoch 1/5  observation 200/4502 batch loss: 2.5338 (avg 2.3498),  avg acc: 0.1169, avg f1: 0.0574, (20.74 im/s)
[TRAIN] epoch 1/5  observation 300/4502 batch loss: 2.2782 (avg 2.3286),  avg acc: 0.1262, avg f1: 0.0612, (20.77 im/s)
[TRAIN] epoch 1/5  observation 400/4502 batch loss: 2.1406 (avg 2.3158),  avg acc: 0.1309, avg f1: 0.0627, (20.93 im/s)
[TRAIN] epoch 1/5  observation 500/4502 batch loss: 2.4447 (avg 2.3055),  avg acc: 0.1381, avg f1: 0.0679, (21.00 im/s)
[TRAIN] epoch 1/5  observation 600/4502 batch loss: 2.3065 (avg 2.2975),  avg acc: 0.1412, avg f1: 0.0695, (21.16 im/s)
[TRAIN] epoch 1/5  observation 700/4502 batch loss: 2.0618 (avg 2.2935),  avg acc: 0.1419, avg f1: 0.0698, (20.89 im/s)
[TRAIN] epoch 1/5  observation 80

...

 60%|██████    | 3/5 [1:11:04<47:27, 1423.65s/it]

Epoch:  3
[TRAIN] epoch 4/5  observation 0/4502 batch loss: 2.1314 (avg 2.1314),  avg acc: 0.1667, avg f1: 0.0556, (17.11 im/s)
[TRAIN] epoch 4/5  observation 100/4502 batch loss: 2.2991 (avg 2.2585),  avg acc: 0.1502, avg f1: 0.0915, (19.89 im/s)
[TRAIN] epoch 4/5  observation 200/4502 batch loss: 2.1932 (avg 2.2542),  avg acc: 0.1451, avg f1: 0.0889, (20.97 im/s)
[TRAIN] epoch 4/5  observation 300/4502 batch loss: 2.1885 (avg 2.2592),  avg acc: 0.1417, avg f1: 0.0876, (20.93 im/s)
[TRAIN] epoch 4/5  observation 400/4502 batch loss: 2.1775 (avg 2.2551),  avg acc: 0.1488, avg f1: 0.0923, (20.98 im/s)
[TRAIN] epoch 4/5  observation 500/4502 batch loss: 2.1440 (avg 2.2550),  avg acc: 0.1494, avg f1: 0.0936, (20.95 im/s)
[TRAIN] epoch 4/5  observation 600/4502 batch loss: 2.2207 (avg 2.2531),  avg acc: 0.1534, avg f1: 0.0967, (21.03 im/s)
[TRAIN] epoch 4/5  observation 700/4502 batch loss: 2.3149 (avg 2.2544),  avg acc: 0.1562, avg f1: 0.0985, (20.85 im/s)
[TRAIN] epoch 4/5  observation 8

 80%|████████  | 4/5 [1:34:56<23:47, 1427.11s/it]

Epoch:  4
[TRAIN] epoch 5/5  observation 0/4502 batch loss: 2.1326 (avg 2.1326),  avg acc: 0.0000, avg f1: 0.0000, (16.83 im/s)
[TRAIN] epoch 5/5  observation 100/4502 batch loss: 1.9978 (avg 2.2594),  avg acc: 0.1452, avg f1: 0.0938, (19.95 im/s)
[TRAIN] epoch 5/5  observation 200/4502 batch loss: 2.2103 (avg 2.2606),  avg acc: 0.1459, avg f1: 0.0934, (20.87 im/s)
[TRAIN] epoch 5/5  observation 300/4502 batch loss: 2.2300 (avg 2.2528),  avg acc: 0.1556, avg f1: 0.1027, (21.09 im/s)
[TRAIN] epoch 5/5  observation 400/4502 batch loss: 2.4344 (avg 2.2535),  avg acc: 0.1608, avg f1: 0.1076, (21.00 im/s)
[TRAIN] epoch 5/5  observation 500/4502 batch loss: 2.1919 (avg 2.2539),  avg acc: 0.1597, avg f1: 0.1078, (20.66 im/s)
[TRAIN] epoch 5/5  observation 600/4502 batch loss: 2.2425 (avg 2.2537),  avg acc: 0.1617, avg f1: 0.1083, (20.64 im/s)
[TRAIN] epoch 5/5  observation 700/4502 batch loss: 2.1646 (avg 2.2540),  avg acc: 0.1591, avg f1: 0.1058, (21.08 im/s)
[TRAIN] epoch 5/5  observation 8

100%|██████████| 5/5 [1:58:50<00:00, 1426.11s/it]


In [ ]:
# # Load rdrsegmenter from VnCoreNLP
# from vncorenlp import VnCoreNLP
# rdrsegmenter = VnCoreNLP("/Absolute-path-to/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word segmentation only
word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

NameError: ignored

In [ ]:
# Input 
text = "Man.United đang tiến rất gần thời điểm trở lại với Champions League, họ chỉ cần giành thêm một điểm trong trận đấu bù với Chelsea vào thứ Năm này là loại bỏ Liverpool ra khỏi cuộc đua. HLV Erik ten Hag cũng không giấu giếm ý đồ muốn tăng cường một tiền đạo đẳng cấp hàng đầu vào mùa hè để cải thiện cho hàng công của mình. Những cái tên như Harry Kane của Tottenham, Victor Osimhen của Napoli và Dusan Vlahovic của Juventus… đã được liên hệ chuyển đến Old Trafford."

# To perform word segmentation only
word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

SyntaxError: ignored